In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import random
import re
from tqdm import tqdm

In [3]:
csv_file = pd.read_csv('../data/index_info.csv')
index_info = csv_file['Index of data without headers'].tolist()

def beautify_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    table = soup.find('table')
    return table

final_df = pd.DataFrame(columns = ['index', 'html_content', 'question', 'answer'])
df = pd.read_csv('../data/dataset_index.csv')

In [36]:
pipe_df = pd.read_csv('../data/confinqa_pipe_original_data.csv')
print(pipe_df.loc[1, 'PIPE_encoding'])

<html>
<body>
  "three factor formula" ) . the consolidated financial statements include northrop grumman management and support services allocations totaling $ 32 million for the year ended december 31 , 2011 . shared services and infrastructure costs - this category includes costs for functions such as information technology support , systems maintenance , telecommunications , procurement and other shared services while hii was a subsidiary of northrop grumman . these costs were generally allocated to the company using the three factor formula or based on usage . the consolidated financial statements reflect shared services and infrastructure costs allocations totaling $ 80 million for the year ended december 31 , 2011 . northrop grumman-provided benefits - this category includes costs for group medical , dental and vision insurance , 401 ( k ) savings plan , pension and postretirement benefits , incentive compensation and other benefits . these costs were generally allocated to the 

In [41]:
html = df.loc[1, 'html_content']
table = beautify_html(html)

In [45]:
results = []

# Extract the data from each row of the table
if table:
    rows = table.find_all('tr')
    for row in rows[1:]:  # Skip the header row
        cells = row.find_all('td')
        data = [cell.get_text(strip=True) for cell in cells]
        results.append(data)
print(results)

[['2', 'sales and service revenues', '$ 1562', '$ 1683', '$ 1637', '$ 1938'], ['3', 'operating income ( loss )', '95', '116', '127', '174'], ['4', 'earnings ( loss ) before income taxes', '65', '87', '99', '143'], ['5', 'net earnings ( loss )', '44', '57', '69', '91'], ['6', 'dividends declared per share', '$ 0.10', '$ 0.10', '$ 0.10', '$ 0.20'], ['7', 'basic earnings ( loss ) per share', '$ 0.88', '$ 1.14', '$ 1.38', '$ 1.86'], ['8', 'diluted earnings ( loss ) per share', '$ 0.87', '$ 1.12', '$ 1.36', '$ 1.82']]


In [38]:

first_row = [td.text for td in table.find('tr').find_all('td')]
# print('first row', first_row, len(first_row))

random_row_index = random.randint(2, len(first_row) - 1)

first_col = [row.find_all('td')[1].text for row in table.find_all('tr')[1:]]
# print('first col', first_col, len(first_col))

if len(first_col) == 1:
    random_col_index = 0
else:
    random_col_index = random.choice(range(len(first_col) - 1))

# print('row random', random_row_index)
# print('col random', random_col_index)

random_col_values = [row.find_all('td')[random_row_index].text for row in table.find_all('tr')[1:]]
# print('col values', random_col_values, len(random_col_values))
# print(first_col[random_col_index])
col_data = first_col[random_col_index].replace('\n', '')
print(col_data)


     dividends declared per share    


In [39]:
print(str(table))

<table class="wikitable">
<tr>
<td>
     1
    </td>
<td>
     ( $ in millions except per share amounts )
    </td>
<td>
     year ended december 31 2013 1st qtr
    </td>
<td>
     year ended december 31 2013 2nd qtr
    </td>
<td>
     year ended december 31 2013 3rd qtr
    </td>
<td>
     year ended december 31 2013 4th qtr
    </td>
</tr>
<tr>
<td>
     2
    </td>
<td>
     sales and service revenues
    </td>
<td>
     $ 1562
    </td>
<td>
     $ 1683
    </td>
<td>
     $ 1637
    </td>
<td>
     $ 1938
    </td>
</tr>
<tr>
<td>
     3
    </td>
<td>
     operating income ( loss )
    </td>
<td>
     95
    </td>
<td>
     116
    </td>
<td>
     127
    </td>
<td>
     174
    </td>
</tr>
<tr>
<td>
     4
    </td>
<td>
     earnings ( loss ) before income taxes
    </td>
<td>
     65
    </td>
<td>
     87
    </td>
<td>
     99
    </td>
<td>
     143
    </td>
</tr>
<tr>
<td>
     5
    </td>
<td>
     net earnings ( loss )
    </td>
<td>
     44
    </td>
<td>
     57
   

In [40]:
# Extracting values for 'cancelled'
pattern = fr'<td>{col_data}</td>(.*?)<\/tr>'
column_values_match = re.search(pattern, str(table), re.DOTALL)
print(column_values_match)

None


In [ ]:

if column_values_match:
    cancelled_values_str = column_values_match.group(1)
    cancelled_values = re.findall(r'-?\d+ \( \d+ \)', cancelled_values_str)
    print(f"What is the value of {col_data}?")
    print(cancelled_values)

In [17]:
for _, row in tqdm(df.iterrows()):
    table = beautify_html(row['html_content'])

    for i in range(50):
        # print('here is index', type(row['index']))
        if int(row['index']) in index_info:
            first_col = [row.find_all('td')[1].text for row in table.find_all('tr')]
            random_index = random.choice(range(len(first_col)))
            second_col = [row.find_all('td')[2].text for row in table.find_all('tr')]
            system_prompt = f"What is {first_col[random_index]}?"
            system_prompt = system_prompt.replace('\n', '')
            answer = second_col[random_index]
        else:
            first_row = [td.text for td in table.find('tr').find_all('td')]
            # print('first row', first_row, len(first_row))

            random_row_index = random.randint(2, len(first_row) - 1)

            first_col = [row.find_all('td')[1].text for row in table.find_all('tr')[1:]]
            # print('first col', first_col, len(first_col))

            if len(first_col) == 1:
                random_col_index = 0
            else:
                random_col_index = random.choice(range(len(first_col) - 1))

            # print('row random', random_row_index)
            # print('col random', random_col_index)

            random_col_values = [row.find_all('td')[random_row_index].text for row in table.find_all('tr')[1:]]
            # print('col values', random_col_values, len(random_col_values))
            # print(first_col[random_col_index])
            col_data = first_col[random_col_index].replace('\n', '')
            row_data = first_row[random_row_index].replace('\n', '')
            system_prompt = f"What is {col_data} for {row_data}?"
            # system_prompt = f"What is {first_col[random_col_index]} for {first_row[random_row_index]}?"

            # print(system_prompt)
            # print(random_col_values[random_col_index].replace('\n', ''))
            # print(random_col_values[random_col_index])
            answer = random_col_values[random_col_index].replace('\n', '')
        system_prompt = re.sub(r'\s+', ' ', system_prompt)
        answer = re.sub(r'\s+', ' ', answer)
        print(system_prompt)
        print(answer)
        print('****************')
        final_df.loc[len(final_df)] = {"index": row['index'], 'html_content': row['html_content'] ,'question': system_prompt, 'answer': answer}
        final_df = final_df.drop_duplicates(subset=['index', 'question', 'answer'])


2it [00:00, 11.21it/s]

What is common stock under the vertex purchase plan ?
 842 
****************
What is common stock under the vertex 401 ( k ) plan ?
 270 
****************
What is total ?
 18851 
****************
What is total ?
 18851 
****************
What is common stock under the vertex purchase plan ?
 842 
****************
What is total ?
 18851 
****************
What is common stock under the vertex purchase plan ?
 842 
****************
What is total ?
 18851 
****************
What is common stock under stock and option plans ?
 17739 
****************
What is common stock under stock and option plans ?
 17739 
****************
What is total ?
 18851 
****************
What is common stock under stock and option plans ?
 17739 
****************
What is common stock under the vertex 401 ( k ) plan ?
 270 
****************
What is common stock under the vertex 401 ( k ) plan ?
 270 
****************
What is total ?
 18851 
****************
What is common stock under stock and option plans ?
 17739

4it [00:00, 12.31it/s]

What is income tax expense for year ended december 31 2009 ( in thousands ) ?
 -1978 ( 1978 ) 
****************
What is gain on sale of discontinued operations net for year ended december 31 2009 ( in thousands ) ?
 12501 
****************
What is income tax expense for year ended december 31 2009 ( in thousands ) ?
 -942 ( 942 ) 
****************
What is gain on sale of discontinued operations before income taxes for year ended december 31 2009 ( in thousands ) ?
 14479 
****************
What is income tax expense for year ended december 31 2009 ( in thousands ) ?
 -1978 ( 1978 ) 
****************
What is gain on sale of discontinued operations before income taxes for year ended december 31 2009 ( in thousands ) ?
 14479 
****************
What is income from discontinued operations net for year ended december 31 2009 ( in thousands ) ?
 4425 
****************
What is gain on sale of discontinued operations before income taxes for year ended december 31 2009 ( in thousands ) ?
 14479 


6it [00:00, 12.92it/s]

...

297it [00:32,  6.90it/s]

What is non-cancelable operating leases net of sublease income for 1-3 years ?
 $ 25.9 
****************
What is 2020 ?
 256172 
****************
What is 2019 ?
 $ 294269 
****************
What is 2021 ?
 210632 
****************
What is 2019 ?
 $ 294269 
****************
What is 2022 ?
 158763 
****************
What is 2021 ?
 210632 
****************
What is 2021 ?
 210632 
****************
What is 2021 ?
 210632 
****************
What is future minimum lease payments ?
 $ 1828519 
****************
What is 2020 ?
 256172 
****************
What is future minimum lease payments ?
 $ 1828519 
****************
What is 2019 ?
 $ 294269 
****************
What is 2021 ?
 210632 
****************
What is 2021 ?
 210632 
****************
What is 2021 ?
 210632 
****************
What is future minimum lease payments ?
 $ 1828519 
****************
What is future minimum lease payments ?
 $ 1828519 
****************
What is 2019 ?
 $ 294269 
****************
What is 2020 ?
 256172 
*************

298it [00:32,  9.07it/s]


What is average loan and lease receivables for 2002 ?
 31.7 
****************
What is automobile origination volume for 2003 ?
 27.8 
****************
What is average loan and lease receivables for change ?
 32 
****************
What is 30+ day delinquency rate for 2003 ?
 1.46 
****************
What is loan and lease receivables for change ?
 16% ( 16 % ) 
****************
What is average loan and lease receivables for change ?
 32 
****************
What is loan and lease receivables for 2003 ?
 $ 43.2 
****************
What is net charge-off ratio for 2003 ?
 0.41 
****************
What is automobile market share for 2003 ?
 6.1% ( 6.1 % ) 
****************
What is 30+ day delinquency rate for 2003 ?
 1.46 
****************
What is average loan and lease receivables for 2002 ?
 31.7 
****************
What is automobile origination volume for change ?
 10 
****************
What is net charge-off ratio for 2003 ?
 0.41 
****************
What is net charge-off ratio for 2002 ?
 0.51 
**

KeyboardInterrupt: 

In [ ]:
# import pdb; pdb.set_trace()
# Remove duplicates based on two columns (Column1 and Column2)
final_df_no_duplicates = final_df.drop_duplicates(subset=['question', 'answer'])
final_df_no_duplicates.to_csv('datasettest1.csv', index = False)